In [ ]:
# ============================
# NOTEBOOK
# Proyecto programación
# Ejecutar en Jupyter
# ============================

# CELDA 1 - Metadatos y objetivo
"""
Autores: Juan Pachecho, Mateo Morales, Juan Camacho
Objetivo: Descargar/limpiar/transformar 3 datasets reales (Superintendencia, DANE GEIH, Exportaciones NME),
exportar 3 CSV limpios listos para Power BI y generar validaciones exigidas por la rúbrica.


# CELDA 2 - Imports y configuración
import os
import pandas as pd
import numpy as np
import requests
import unicodedata
from io import BytesIO
import zipfile
import warnings
warnings.filterwarnings("ignore")

os.makedirs("data/raw", exist_ok=True)
os.makedirs("data/clean", exist_ok=True)

# CELDA 3 - Helpers
def normalize_str_col(s):
    if pd.isna(s):
        return s
    x = str(s).strip().upper()
    x = unicodedata.normalize("NFKD", x).encode("ascii", "ignore").decode("utf-8")
    x = " ".join(x.split())
    return x

def detect_column(df, candidates):
    for c in candidates:
        if c in df.columns:
            return c
    return None

def safe_read_csv(path_or_url, **kwargs):
    # intenta leer CSV local o URL
    try:
        return pd.read_csv(path_or_url, low_memory=False, **kwargs)
    except Exception as e:
        raise RuntimeError(f"No fue posible leer {path_or_url}: {e}")

# CELDA 4 - Rutas / URLs (ajusta si necesitas descargar manualmente)
# Si no funcionan las descargas automáticas, descarga manualmente y coloca archivos en data/raw/
# Ejemplos de placeholders (reemplaza por links reales si quieres descarga automática)
DOWNLOAD_URL_EXPORT = "data/raw/exportaciones_nme.csv"  # ya tienes el CSV local según tu ejecución
DOWNLOAD_URL_SUPER = "data/raw/supersociedades.csv"      # si lo descargaste manualmente
DOWNLOAD_URL_GEIH = "data/raw/geih.csv"                  # si lo descargaste manualmente

# CELDA 5 - Carga de datos (ajusta rutas si guardaste en otra carpeta)
print("Leyendo exportaciones desde:", DOWNLOAD_URL_EXPORT)
export_df = safe_read_csv(DOWNLOAD_URL_EXPORT)

# Intenta leer Supersociedades y GEIH si existen (si no, se generan placeholders vacíos)
if os.path.exists(DOWNLOAD_URL_SUPER):
    sup_df = safe_read_csv(DOWNLOAD_URL_SUPER)
else:
    sup_df = pd.DataFrame()
    print("Archivo Supersociedades no encontrado en data/raw/. Si no lo tienes, descarga manualmente y coloca en data/raw/supersociedades.csv")

if os.path.exists(DOWNLOAD_URL_GEIH):
    geih_df = safe_read_csv(DOWNLOAD_URL_GEIH)
else:
    geih_df = pd.DataFrame()
    print("Archivo GEIH no encontrado en data/raw/. Si no lo tienes, descarga manualmente y coloca en data/raw/geih.csv")

# CELDA 6 - Mostrar columnas (útil para depuración)
print("Columnas export_df:", export_df.columns.tolist())
print("Columnas supersociedades (si existen):", sup_df.columns.tolist() if not sup_df.empty else "No hay Supersociedades")
print("Columnas GEIH (si existen):", geih_df.columns.tolist() if not geih_df.empty else "No hay GEIH")

# CELDA 7 - Normalizar nombres básicos de export_df
# Detectar columna de fecha y convertir a datetime -> extraer year
date_candidates = ["fecha", "fecha_de_corte", "fecha_corte", "date", "fecha_registro"]
date_col = detect_column(export_df, date_candidates)

if date_col:
    try:
        export_df[date_col] = pd.to_datetime(export_df[date_col], errors="coerce")
        export_df["year"] = export_df[date_col].dt.year
        print(f"Usando columna de fecha '{date_col}' para extraer 'year'.")
    except Exception as e:
        print("No se pudo parsear fecha en columna", date_col, ":", e)
        export_df["year"] = pd.NA
else:
    # Si no hay fecha, intenta detectar columna 'anio'/'ano'/'year'
    year_candidates = [c for c in export_df.columns if c.lower() in ("anio","año","ano","year")]
    if year_candidates:
        export_df.rename(columns={year_candidates[0]:"year"}, inplace=True)
        export_df["year"] = pd.to_numeric(export_df["year"], errors="coerce").astype("Int64")
        print(f"Usando columna '{year_candidates[0]}' como year.")
    else:
        print("No se detectó columna de fecha ni year en export_df. Creando year por defecto NA.")
        export_df["year"] = pd.NA

# CELDA 8 - Detectar columna de valor de exportaciones (varios nombres posibles)
valor_candidates = [
    "valor_exportaciones_nme",
    "valor_exportaciones_totales",
    "valor_exportaciones_nme_1",
    "valor_de_exportaciones_totales",
    "valor_exportaciones_n",
    "valor_export_usd",
    "valor_fob_usd",
    "valor"
]
valor_col = detect_column(export_df, valor_candidates)
if valor_col is None:
    # intenta detectar columnas numéricas que contengan 'export' y 'valor'
    for c in export_df.columns:
        if "export" in c.lower() and ("valor" in c.lower() or "fob" in c.lower() or "usd" in c.lower()):
            valor_col = c
            break

if valor_col is None:
    # fallback: intenta detectar la primera columna numérica grande
    numeric_cols = export_df.select_dtypes(include=[np.number]).columns.tolist()
    if numeric_cols:
        valor_col = numeric_cols[0]
        print(f"No se encontró columna específica de valor; se usará '{valor_col}' como fallback.")
    else:
        raise ValueError("No se pudo identificar columna de valor en export_df. Columnas: " + ", ".join(export_df.columns))

# Normalizar la columna de valor a float
export_df[valor_col] = pd.to_numeric(export_df[valor_col], errors="coerce").fillna(0)
export_df = export_df.rename(columns={valor_col: "valor_export_usd"})
print(f"Usando columna '{valor_col}' como 'valor_export_usd'.")

# CELDA 9 - Detectar columna departamento (si existe)
dept_candidates = [
    "departamento", "departamento_nom", "dpto", "departamento_dane", "departamento_exportador",
    "departamento_origen", "dpto_nom", "departamento_nombre", "depto"
]
dept_col = detect_column(export_df, dept_candidates)
if dept_col:
    export_df[dept_col] = export_df[dept_col].apply(normalize_str_col)
    export_df.rename(columns={dept_col:"departamento"}, inplace=True)
    print(f"Columna departamento detectada: {dept_col} -> renombrada a 'departamento'.")
else:
    print("No se detectó columna 'departamento' en export_df. El agrupamiento se hará por 'year' únicamente.")

# CELDA 10 - Agregación exportaciones por (departamento?, year)
if "departamento" in export_df.columns and export_df["departamento"].notna().sum() > 0:
    export_agg = (
        export_df.groupby(["departamento","year"], dropna=False)
        .agg(valor_export_usd=("valor_export_usd","sum"))
        .reset_index()
    )
else:
    # Si no hay departamento, agregamos por year solamente
    export_agg = (
        export_df.groupby(["year"], dropna=False)
        .agg(valor_export_usd=("valor_export_usd","sum"))
        .reset_index()
    )
    # para compatibilidad con el pipeline, creamos columna departamento = 'SIN_DEPT' si la usamos luego
    export_agg["departamento"] = "SIN_DEPT"
    # Reordenar columnas para consistencia
    export_agg = export_agg[["departamento","year","valor_export_usd"]]

print("Exportaciones agregadas shape:", export_agg.shape)
display(export_agg.head(5))

# CELDA 11 - Limpieza Supersociedades (si se cargó)
if not sup_df.empty:
    # Normalizar nombres de columna más comunes
    sup_df.columns = [c.strip() for c in sup_df.columns]
    # Detectar columnas relevantes
    sup_dept = detect_column(sup_df, dept_candidates)
    sup_nit = detect_column(sup_df, ["NIT","nit","Nit","nit_empresa"])
    sup_anio = detect_column(sup_df, [c for c in sup_df.columns if c.lower() in ("anio","año","ano","year","periodo")])
    sup_ing = detect_column(sup_df, ["INGRESOS","ingresos","ingreso","total_ingresos","ventas"])
    sup_act = detect_column(sup_df, ["ACTIVOS","activos","activo"])
    sup_util = detect_column(sup_df, ["UTILIDAD_OPERATIVA","utilidad_operativa","utilidad","resultado_operativo"])
    # Normalizaciones y renombres conservadores
    rename_map = {}
    if sup_dept: rename_map[sup_dept] = "departamento"
    if sup_anio: rename_map[sup_anio] = "year"
    if sup_ing: rename_map[sup_ing] = "ingresos"
    if sup_act: rename_map[sup_act] = "activos"
    if sup_util: rename_map[sup_util] = "utilidad_operativa"
    if sup_nit: rename_map[sup_nit] = "NIT"
    sup_df = sup_df.rename(columns=rename_map)
    # Normalizar departamento si existe
    if "departamento" in sup_df.columns:
        sup_df["departamento"] = sup_df["departamento"].apply(normalize_str_col)
    # Forzar tipos numericos
    for numcol in ["ingresos","activos","utilidad_operativa","year"]:
        if numcol in sup_df.columns:
            sup_df[numcol] = pd.to_numeric(sup_df[numcol], errors="coerce")
    # Agregación por departamento+year si hay departamento, si no, por year y poner 'SIN_DEPT'
    if "departamento" in sup_df.columns and sup_df["departamento"].notna().sum() > 0:
        finanzas_dept = sup_df.groupby(["departamento","year"], dropna=False).agg({
            "ingresos":"sum",
            "utilidad_operativa":"sum",
            "activos":"sum"
        }).reset_index()
    else:
        temp = sup_df.groupby(["year"], dropna=False).agg({
            "ingresos":"sum",
            "utilidad_operativa":"sum",
            "activos":"sum"
        }).reset_index()
        temp["departamento"] = "SIN_DEPT"
        finanzas_dept = temp[["departamento","year","ingresos","utilidad_operativa","activos"]]
    print("Finanzas aggregated shape:", finanzas_dept.shape)
    display(finanzas_dept.head(5))
else:
    # Si no tienes Supersociedades local, crear archivo vacío para no romper pipeline (debes descargarlo)
    finanzas_dept = pd.DataFrame(columns=["departamento","year","ingresos","utilidad_operativa","activos"])
    print("Supersociedades no cargado - finanzas_dept vacío creado. Descarga el CSV si quieres usar datos empresariales reales.")

# CELDA 12 - Limpieza GEIH (si se cargó)
if not geih_df.empty:
    # Normalizar columnas
    geih_df.columns = [c.strip() for c in geih_df.columns]
    # Detectar campos relevantes
    geih_dept = detect_column(geih_df, dept_candidates)
    geih_year = detect_column(geih_df, [c for c in geih_df.columns if c.lower() in ("ano","año","ano_periodo","year","ano_encuesta","ano")])
    geih_ocup = detect_column(geih_df, ["OCUPADO","ocupado","ocupados","personas_ocupadas"])
    geih_ing = detect_column(geih_df, ["INGRESO","ingreso","ingresos","ing_med"])
    rename_map = {}
    if geih_dept: rename_map[geih_dept] = "departamento"
    if geih_year: rename_map[geih_year] = "year"
    if geih_ocup: rename_map[geih_ocup] = "ocupado"
    if geih_ing: rename_map[geih_ing] = "ingreso"
    geih_df = geih_df.rename(columns=rename_map)
    if "departamento" in geih_df.columns:
        geih_df["departamento"] = geih_df["departamento"].apply(normalize_str_col)
    # Forzar tipos
    if "year" in geih_df.columns:
        geih_df["year"] = pd.to_numeric(geih_df["year"], errors="coerce").astype("Int64")
    if "ocupado" in geih_df.columns:
        geih_df["ocupado"] = pd.to_numeric(geih_df["ocupado"], errors="coerce").fillna(0)
    if "ingreso" in geih_df.columns:
        geih_df["ingreso"] = pd.to_numeric(geih_df["ingreso"], errors="coerce").fillna(0)
    # Agregación
    if "departamento" in geih_df.columns and geih_df["departamento"].notna().sum() > 0:
        geih_agg = geih_df.groupby(["departamento","year"], dropna=False).agg({
            "ocupado":"sum",
            "ingreso":"mean"
        }).reset_index()
    else:
        temp = geih_df.groupby(["year"], dropna=False).agg({
            "ocupado":"sum" if "ocupado" in geih_df.columns else (lambda x: 0),
            "ingreso":"mean" if "ingreso" in geih_df.columns else (lambda x: 0)
        }).reset_index()
        temp["departamento"] = "SIN_DEPT"
        geih_agg = temp[["departamento","year","ocupado","ingreso"]]
    print("GEIH aggregated shape:", geih_agg.shape)
    display(geih_agg.head(5))
else:
    geih_agg = pd.DataFrame(columns=["departamento","year","ocupado","ingreso"])
    print("GEIH no cargado - geih_agg vacío creado. Descarga el microdato si quieres usar empleo real.")

# CELDA 13 - Merge final por departamento+year
# Queremos una tabla de hechos por departamento+year que contenga: ingresos, utilidad_operativa, activos, valor_export_usd, ocupado, ingreso_promedio
# Empezamos desde finanzas_dept (si está vacío, partimos de export_agg)
if not finanzas_dept.empty:
    df_merge = finanzas_dept.merge(export_agg, on=["departamento","year"], how="left")
else:
    df_merge = export_agg.copy()

# Merge con geih_agg si existe
if not geih_agg.empty:
    df_merge = df_merge.merge(geih_agg, on=["departamento","year"], how="left")

# Rellenar NA's razonablemente
for col in ["valor_export_usd","ocupado","ingreso","ingresos","utilidad_operativa","activos"]:
    if col in df_merge.columns:
        if df_merge[col].dtype.kind in 'biufc':
            df_merge[col] = df_merge[col].fillna(0)
        else:
            df_merge[col] = df_merge[col].fillna(0)

# CELDA 14 - KPIs calculados
# Evitar división por cero
df_merge["margen_operativo"] = df_merge.apply(lambda r: (r["utilidad_operativa"]/r["ingresos"]) if (r.get("ingresos",0) and r["ingresos"]!=0) else np.nan, axis=1)
df_merge["export_sobre_ingresos"] = df_merge.apply(lambda r: (r["valor_export_usd"]/r["ingresos"]) if (r.get("ingresos",0) and r["ingresos"]!=0) else np.nan, axis=1)

# CELDA 15 - Validaciones exigidas por la rúbrica
# 1) Asegurar que existen 3 datasets limpios
assert "departamento" in export_agg.columns and "year" in export_agg.columns, "Exportaciones agregadas no tienen departamento+year"
assert "departamento" in df_merge.columns and "year" in df_merge.columns, "Merge final no tiene departamento+year"

# 2) Revisiones rápidas
print("Merge final shape:", df_merge.shape)
display(df_merge.head(6))

# CELDA 16 - Guardar CSVs limpios para Power BI
finanzas_dept.to_csv("data/clean/finanzas_clean.csv", index=False)
export_agg.to_csv("data/clean/exportaciones_clean.csv", index=False)
geih_agg.to_csv("data/clean/empleo_clean.csv", index=False)
df_merge.to_csv("data/clean/merge_dashboard.csv", index=False)
print("CSV limpios guardados en data/clean/:", os.listdir("data/clean"))

# CELDA 17 - Mensajes finales y recomendaciones (markdown style)
print("""
Hecho:
- Se generaron CSVs limpios: finanzas_clean.csv, exportaciones_clean.csv, empleo_clean.csv y merge_dashboard.csv.
- Revisa 'data/clean' y valida que 'departamento' esté uniforme (sin tildes/variantes).
- Si Supersociedades o GEIH no se cargaron porque no estaban en data/raw/, descarga manualmente y re-ejecuta las celdas correspondientes.
Pasos siguientes (Power BI): abrir Power BI, importar los 3 CSV (finanzas_clean, empleo_clean, exportaciones_clean),
crear relaciones por departamento+year y pegar las medidas DAX que te entregué en el repo.
""")


Leyendo exportaciones desde: data/raw/exportaciones_nme.csv
Archivo Supersociedades no encontrado en data/raw/. Si no lo tienes, descarga manualmente y coloca en data/raw/supersociedades.csv
Archivo GEIH no encontrado en data/raw/. Si no lo tienes, descarga manualmente y coloca en data/raw/geih.csv
Columnas export_df: ['fecha', 'fecha_de_corte', 'valor_exportaciones_totales', 'valor_exportaciones_nme', 'exportaciones_de_servicios', 'valor_exportaciones_nme_1', 'valor_de_exportaciones_totales', 'valor_exportaciones_nme_2']
Columnas supersociedades (si existen): No hay Supersociedades
Columnas GEIH (si existen): No hay GEIH
Usando columna de fecha 'fecha' para extraer 'year'.
Usando columna 'valor_exportaciones_nme' como 'valor_export_usd'.
No se detectó columna 'departamento' en export_df. El agrupamiento se hará por 'year' únicamente.
Exportaciones agregadas shape: (26, 3)
departamento	year	valor_export_usd
0	SIN_DEPT	2000	7073
1	SIN_DEPT	2001	7398
2	SIN_DEPT	2002	7107
3	SIN_DEPT	2003	7037
4	SIN_DEPT	2004	9153
Supersociedades no cargado - finanzas_dept vacío creado. Descarga el CSV si quieres usar datos empresariales reales.
GEIH no cargado - geih_agg vacío creado. Descarga el microdato si quieres usar empleo real.
Merge final shape: (26, 5)
departamento	year	valor_export_usd	margen_operativo	export_sobre_ingresos
0	SIN_DEPT	2000	7073	NaN	NaN
1	SIN_DEPT	2001	7398	NaN	NaN
2	SIN_DEPT	2002	7107	NaN	NaN
3	SIN_DEPT	2003	7037	NaN	NaN
4	SIN_DEPT	2004	9153	NaN	NaN
5	SIN_DEPT	2005	11253	NaN	NaN
CSV limpios guardados en data/clean/: ['finanzas_clean.csv', 'empleo_clean.csv', 'merge_dashboard.csv', 'exportaciones_clean.csv']

Hecho:
- Se generaron CSVs limpios: finanzas_clean.csv, exportaciones_clean.csv, empleo_clean.csv y merge_dashboard.csv.
- Revisa 'data/clean' y valida que 'departamento' esté uniforme (sin tildes/variantes).
- Si Supersociedades o GEIH no se cargaron porque no estaban en data/raw/, descarga manualmente y re-ejecuta las celdas correspondientes.
Pasos siguientes (Power BI): abrir Power BI, importar los 3 CSV (finanzas_clean, empleo_clean, exportaciones_clean),
crear relaciones por departamento+year y pegar las medidas DAX que te entregué en el repo.


Leyendo exportaciones desde: data/raw/exportaciones_nme.csv
Archivo Supersociedades no encontrado en data/raw/. Si no lo tienes, descarga manualmente y coloca en data/raw/supersociedades.csv
Archivo GEIH no encontrado en data/raw/. Si no lo tienes, descarga manualmente y coloca en data/raw/geih.csv
Columnas export_df: ['fecha', 'fecha_de_corte', 'valor_exportaciones_totales', 'valor_exportaciones_nme', 'exportaciones_de_servicios', 'valor_exportaciones_nme_1', 'valor_de_exportaciones_totales', 'valor_exportaciones_nme_2']
Columnas supersociedades (si existen): No hay Supersociedades
Columnas GEIH (si existen): No hay GEIH
Usando columna de fecha 'fecha' para extraer 'year'.
Usando columna 'valor_exportaciones_nme' como 'valor_export_usd'.
No se detectó columna 'departamento' en export_df. El agrupamiento se hará por 'year' únicamente.
Exportaciones agregadas shape: (26, 3)


,departamento,year,valor_export_usd
0,SIN_DEPT,2000,7073
1,SIN_DEPT,2001,7398
2,SIN_DEPT,2002,7107
3,SIN_DEPT,2003,7037
4,SIN_DEPT,2004,9153


Supersociedades no cargado - finanzas_dept vacío creado. Descarga el CSV si quieres usar datos empresariales reales.
GEIH no cargado - geih_agg vacío creado. Descarga el microdato si quieres usar empleo real.
Merge final shape: (26, 5)


,departamento,year,valor_export_usd,margen_operativo,export_sobre_ingresos
0,SIN_DEPT,2000,7073,NaN,NaN
1,SIN_DEPT,2001,7398,NaN,NaN
2,SIN_DEPT,2002,7107,NaN,NaN
3,SIN_DEPT,2003,7037,NaN,NaN
4,SIN_DEPT,2004,9153,NaN,NaN
5,SIN_DEPT,2005,11253,NaN,NaN


CSV limpios guardados en data/clean/: ['finanzas_clean.csv', 'exportaciones_clean.xlsx', 'finanzas_clean.xlsx', 'empleo_clean.csv', 'empleo_clean.xlsx', 'merge_dashboard.csv', 'exportaciones_clean.csv']


...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>